In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## OutputParser

언어 모델은 텍스트를 출력합니다. 그러나 많은 경우 단순한 텍스트보다 더 구조화된 정보를 얻고 싶을 수 있습니다. 이때 `OutputParser` 가 필요합니다.

`OutputParser` 는 언어 모델 응답을 구조화하는 데 도움이 되는 클래스입니다.

`OutputParser` 가 구현해야 하는 두 가지 주요 메서드가 있습니다:

- **"Get format instructions"** : 언어 모델의 출력 형식을 지정하는 방법에 대한 지침이 포함된 문자열을 반환하는 메서드입니다.
- **"Parse"** : 문자열(언어 모델의 응답으로 가정)을 받아 특정 구조로 구문 분석하는 메서드입니다.

그리고 하나의 선택적 메서드가 있습니다:

- **"Parse with prompt"** : 문자열(언어 모델의 응답으로 가정)과 프롬프트(해당 응답을 생성한 프롬프트로 가정)를 받아 특정 구조로 파싱하는 메서드입니다. 프롬프트는 주로 `OutputParser` 가 어떤 식으로든 출력을 다시 시도하거나 수정하려는 경우에 제공되며, 이를 위해 프롬프트의 정보가 필요합니다.


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

In [32]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [46]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=EmailSummary)
parser

PydanticOutputParser(pydantic_object=<class '__main__.EmailSummary'>)

In [58]:
query = "다음의 이메일 내용중 에서 필요한 정보를 발췌해 주세요."

parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [59]:
prompt = PromptTemplate(
    template="유저의 질문에 답변해 주세요.\n{format_instructions}\n{query}\n이메일 내용:\n{context}\n\n답변:",
    input_variables=["query", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['context', 'query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"title": "Person", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c\\uc758 \\uc774\\uba54\\uc77c \\uc8fc\\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\\uba54\\uc77c \\uc81c\\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\\u

In [60]:
chain = prompt | llm

In [61]:
response = chain.invoke({"query": query, "context": email_conversation})
print(response.content)

{
  "person": "이은채",
  "email": "eunchae@teddyinternational.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.",
  "date": "다음 주 화요일(1월 15일) 오전 10시"
}


In [62]:
email_object = parser.parse(response.content)
email_object

EmailSummary(person='이은채', email='eunchae@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.', date='다음 주 화요일(1월 15일) 오전 10시')

In [64]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

    def __str__(self):
        return f"{self.person}({self.email})님으로부터 온 메일\n{self.subject}\n\n{self.summary}\n\n{self.date}에 미팅이 예정되어 있습니다."


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=EmailSummary)
parser

PydanticOutputParser(pydantic_object=<class '__main__.EmailSummary'>)

In [65]:
email_object = parser.parse(response.content)
email_object

EmailSummary(person='이은채', email='eunchae@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.', date='다음 주 화요일(1월 15일) 오전 10시')

In [66]:
print(email_object)

이은채(eunchae@teddyinternational.me)님으로부터 온 메일
"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

다음 주 화요일(1월 15일) 오전 10시에 미팅이 예정되어 있습니다.


필요한 정보가 메일 본문에 없는 경우


In [51]:
email_conversation2 = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [52]:
response = chain.invoke({"query": query, "context": email_conversation2})
print(response.content)

{
  "person": "이은채",
  "email": "eunchae@teddyinternational.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.",
  "date": "N/A"
}
